# 🎤 CLARISSA Voice Input Showcase

**Talk to Your Reservoir Simulation**

This notebook demonstrates CLARISSA's voice interface with **professional waveform visualization** - control reservoir simulations through natural speech.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/wolfram-laube/clarissa/blob/main/docs/tutorials/notebooks/16_Voice_Input_Showcase.ipynb)

---

## ✨ Features

- **Real-time Waveform Animation** - 30-bar WebAudio visualizer
- **Speech-to-Text** - OpenAI Whisper integration
- **Intent Recognition** - LLM-based command parsing
- **3D Visualization** - Plotly reservoir property displays

---

## 1️⃣ Setup & Installation

In [ ]:
# Install required packages
!pip install -q openai anthropic plotly numpy ipywidgets

print("✅ Packages installed")

In [ ]:
import os
import json
import base64
import tempfile
from dataclasses import dataclass, field
from typing import Dict, Any, Optional, List
from enum import Enum
import numpy as np
import plotly.graph_objects as go
from IPython.display import display, HTML, Audio, Javascript
import ipywidgets as widgets

print("✅ Imports ready")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🔑 API Key Setup
# ═══════════════════════════════════════════════════════════════════════════════

# Option 1: Colab secrets (recommended)
try:
    from google.colab import userdata
    
    try:
        os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
        print("✅ OpenAI API key loaded from Colab secrets")
    except:
        pass
        
    try:
        os.environ['ANTHROPIC_API_KEY'] = userdata.get('ANTHROPIC_API_KEY')
        print("✅ Anthropic API key loaded from Colab secrets")
    except:
        pass
        
except ImportError:
    print("ℹ️  Not running in Colab - set API keys manually")

# Check what's available
openai_key = os.getenv('OPENAI_API_KEY')
anthropic_key = os.getenv('ANTHROPIC_API_KEY')

print()
print("═" * 50)
print("Configuration Status:")
print("═" * 50)

if openai_key:
    print("🟢 Whisper STT: Available")
    print("🟢 GPT-4 Intent Parsing: Available")
else:
    print("🟡 Whisper STT: Not available (no OpenAI key)")
    print("   → Text input mode only")

if anthropic_key:
    print("🟢 Claude Intent Parsing: Available")

print("═" * 50)
print()
print("💡 To add API keys in Colab:")
print("   1. Click 🔑 Secrets (left sidebar)")
print("   2. Add: OPENAI_API_KEY")
print("   3. Enable notebook access")
print("   4. Re-run this cell")

## 2️⃣ Core Components

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# Intent Types & Data Classes
# ═══════════════════════════════════════════════════════════════════════════════

class IntentType(Enum):
    VISUALIZE_PROPERTY = "visualize_property"
    QUERY_VALUE = "query_value"
    NAVIGATE = "navigate"
    HELP = "help"
    CANCEL = "cancel"
    CONFIRM = "confirm"
    UNKNOWN = "unknown"

@dataclass
class Intent:
    type: IntentType
    confidence: float
    slots: Dict[str, Any] = field(default_factory=dict)
    raw_text: str = ""

@dataclass
class VoiceResponse:
    success: bool
    text: str
    intent: Optional[Intent] = None
    visualization: Optional[go.Figure] = None

print("✅ Data classes defined")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# Demo Reservoir Data
# ═══════════════════════════════════════════════════════════════════════════════

NX, NY, NZ = 10, 10, 5

def generate_demo_data():
    """Generate synthetic reservoir properties."""
    np.random.seed(42)
    
    # Permeability with channel (high-perm streak)
    perm = np.random.lognormal(5, 1, (NX, NY, NZ))
    for j in range(3, 7):
        perm[4:7, j, :] *= 5  # High-perm channel
    
    # Porosity correlated with permeability
    poro = 0.1 + 0.2 * (np.log(perm) - np.log(perm).min()) / (np.log(perm).max() - np.log(perm).min())
    poro += np.random.normal(0, 0.02, poro.shape)
    poro = np.clip(poro, 0.05, 0.35)
    
    # Water saturation (waterflood front)
    sw = np.zeros((NX, NY, NZ))
    for i in range(NX):
        sw[i, :, :] = 0.2 + 0.6 * (i / NX)
    sw += np.random.normal(0, 0.05, sw.shape)
    sw = np.clip(sw, 0.2, 0.8)
    
    # Pressure declining toward producer
    pressure = np.zeros((NX, NY, NZ))
    for i in range(NX):
        for j in range(NY):
            pressure[i, j, :] = 4000 - 500 * np.sqrt((i/NX)**2 + (j/NY)**2)
    
    return {
        'permeability': perm,
        'porosity': poro,
        'water_saturation': sw,
        'pressure': pressure
    }

DEMO_DATA = generate_demo_data()
print("✅ Demo reservoir data generated (10×10×5 grid)")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# Intent Parser (Rule-based + LLM fallback)
# ═══════════════════════════════════════════════════════════════════════════════
import re

def parse_intent(text: str) -> Intent:
    """Parse natural language into structured intent."""
    text_lower = text.lower().strip()
    slots = {}
    
    # Cancel/Stop
    if any(kw in text_lower for kw in ["cancel", "stop", "abort", "quit", "nevermind"]):
        return Intent(IntentType.CANCEL, 1.0, {}, text)
    
    # Confirm
    if text_lower in ["yes", "yeah", "yep", "confirm", "ok", "okay", "do it", "go ahead"]:
        return Intent(IntentType.CONFIRM, 1.0, {}, text)
    
    # Help
    if text_lower == "help" or "what can" in text_lower or "how do" in text_lower:
        return Intent(IntentType.HELP, 1.0, {}, text)
    
    # Visualization
    viz_keywords = ["show", "display", "visualize", "plot", "view", "see"]
    if any(kw in text_lower for kw in viz_keywords):
        # Extract property
        if "perm" in text_lower:
            slots["property"] = "permeability"
        elif "poro" in text_lower:
            slots["property"] = "porosity"
        elif "saturation" in text_lower or " sw" in text_lower or "water sat" in text_lower:
            slots["property"] = "water_saturation"
        elif "pressure" in text_lower:
            slots["property"] = "pressure"
        
        # Extract layer
        layer_match = re.search(r'layer\s*(\d+)', text_lower)
        if layer_match:
            slots["layer"] = int(layer_match.group(1))
        
        # Extract time
        time_match = re.search(r'(?:day|time|t=?)\s*(\d+)', text_lower)
        if time_match:
            slots["time_days"] = int(time_match.group(1))
        
        # Default to permeability if no property specified
        if not slots.get("property") and not slots.get("layer"):
            slots["property"] = "permeability"
        
        return Intent(IntentType.VISUALIZE_PROPERTY, 0.95, slots, text)
    
    # Query
    if any(kw in text_lower for kw in ["what", "how much", "tell me", "get"]):
        if "oil rate" in text_lower or "fopr" in text_lower:
            slots["property"] = "oil_rate"
        elif "water cut" in text_lower or "wct" in text_lower:
            slots["property"] = "water_cut"
        elif "pressure" in text_lower or "bhp" in text_lower:
            slots["property"] = "pressure"
        elif "gor" in text_lower or "gas oil" in text_lower:
            slots["property"] = "gor"
        elif "cumulative" in text_lower or "total" in text_lower:
            slots["property"] = "cumulative_oil"
        else:
            slots["property"] = "value"
        
        return Intent(IntentType.QUERY_VALUE, 0.9, slots, text)
    
    return Intent(IntentType.UNKNOWN, 0.3, {}, text)

# Test
test_result = parse_intent("show me the permeability")
print(f"✅ Intent parser ready")
print(f"   Test: 'show me the permeability' → {test_result.type.value}, slots={test_result.slots}")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# Visualization Functions
# ═══════════════════════════════════════════════════════════════════════════════

def create_3d_visualization(prop_name: str, prop_data: np.ndarray) -> go.Figure:
    """Create 3D scatter plot of property."""
    x, y, z, values = [], [], [], []
    
    for i in range(NX):
        for j in range(NY):
            for k in range(NZ):
                x.append(i)
                y.append(j)
                z.append(k)
                values.append(prop_data[i, j, k])
    
    # Color scale based on property
    colorscale = 'Viridis' if prop_name != 'pressure' else 'RdYlBu'
    
    fig = go.Figure(data=[go.Scatter3d(
        x=x, y=y, z=z,
        mode='markers',
        marker=dict(
            size=8,
            color=values,
            colorscale=colorscale,
            colorbar=dict(title=prop_name.replace('_', ' ').title()),
            opacity=0.8
        )
    )])
    
    fig.update_layout(
        title=f"3D {prop_name.replace('_', ' ').title()} Distribution",
        scene=dict(
            xaxis_title='I',
            yaxis_title='J',
            zaxis_title='K'
        ),
        template='plotly_dark',
        height=500
    )
    
    return fig

def create_cross_section(prop_name: str, prop_data: np.ndarray, layer: int) -> go.Figure:
    """Create 2D heatmap at specified layer."""
    layer = max(1, min(layer, NZ)) - 1  # Convert to 0-indexed
    
    fig = go.Figure(data=go.Heatmap(
        z=prop_data[:, :, layer],
        colorscale='Viridis',
        colorbar=dict(title=prop_name.replace('_', ' ').title())
    ))
    
    fig.update_layout(
        title=f"{prop_name.replace('_', ' ').title()} - Layer {layer + 1}",
        xaxis_title='J',
        yaxis_title='I',
        template='plotly_dark',
        height=450
    )
    
    return fig

print("🎨 Visualization functions ready")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# Command Executor
# ═══════════════════════════════════════════════════════════════════════════════

def execute_intent(intent: Intent) -> VoiceResponse:
    """Execute parsed intent and return response."""
    slots = intent.slots
    
    if intent.type == IntentType.VISUALIZE_PROPERTY:
        prop = slots.get('property', 'permeability')
        layer = slots.get('layer')
        
        # Get data
        if prop in DEMO_DATA:
            prop_data = DEMO_DATA[prop]
            prop_name = prop
        else:
            prop_data = DEMO_DATA['permeability']
            prop_name = 'permeability'
        
        # Create visualization
        if layer:
            fig = create_cross_section(prop_name, prop_data, layer)
            text = f"Showing {prop_name.replace('_', ' ')} at layer {layer}."
        else:
            fig = create_3d_visualization(prop_name, prop_data)
            text = f"Showing {prop_name.replace('_', ' ')} in 3D."
        
        return VoiceResponse(True, text, intent, fig)
    
    elif intent.type == IntentType.QUERY_VALUE:
        prop = slots.get('property', 'value')
        
        # Simulated values
        values = {
            'oil_rate': '2,450 STB/day',
            'water_cut': '35%',
            'pressure': '3,200 psi',
            'gor': '850 scf/STB',
            'cumulative_oil': '1.2 MMSTB'
        }
        
        value = values.get(prop, 'Unknown')
        return VoiceResponse(True, f"The {prop.replace('_', ' ')} is {value}.", intent)
    
    elif intent.type == IntentType.HELP:
        help_text = """Available commands:
• "Show permeability" - 3D property view
• "Show layer 3" - Cross-section at layer
• "What is the water cut?" - Query values
• "Show saturation at day 500" - Time-specific view"""
        return VoiceResponse(True, help_text, intent)
    
    elif intent.type == IntentType.CANCEL:
        return VoiceResponse(True, "Operation cancelled.", intent)
    
    elif intent.type == IntentType.CONFIRM:
        return VoiceResponse(True, "Confirmed! Executing...", intent)
    
    else:
        return VoiceResponse(False, "I didn't understand. Try 'show permeability' or 'help'.", intent)

print("⚡ Command executor ready")

---

## 3️⃣ 🎤 Voice Recording with Waveform Visualization

**Professional recording interface with real-time audio visualization!**

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🎤 PROFESSIONAL VOICE RECORDER WITH WAVEFORM VISUALIZATION
# ═══════════════════════════════════════════════════════════════════════════════

from IPython.display import display, Javascript, HTML
from google.colab.output import eval_js
from base64 import b64decode

VOICE_RECORDER_HTML = '''
<style>
    .clarissa-recorder {
        font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', sans-serif;
        background: linear-gradient(135deg, #1a1a2e 0%, #16213e 100%);
        border-radius: 16px;
        padding: 30px;
        max-width: 500px;
        margin: 20px auto;
        box-shadow: 0 10px 40px rgba(0,0,0,0.4);
        color: white;
        text-align: center;
    }
    
    .clarissa-recorder h2 {
        margin: 0 0 5px 0;
        font-size: 1.5em;
        background: linear-gradient(90deg, #e94560, #0f3460);
        -webkit-background-clip: text;
        -webkit-text-fill-color: transparent;
    }
    
    .clarissa-recorder .subtitle {
        color: #888;
        margin-bottom: 25px;
        font-size: 0.95em;
    }
    
    .waveform-container {
        height: 80px;
        background: rgba(0,0,0,0.3);
        border-radius: 12px;
        display: flex;
        align-items: center;
        justify-content: center;
        gap: 3px;
        padding: 0 20px;
        margin: 20px 0;
    }
    
    .wave-bar {
        width: 4px;
        height: 20px;
        background: linear-gradient(180deg, #e94560, #0f3460);
        border-radius: 2px;
        transition: height 0.05s ease-out;
    }
    
    .wave-bar.active {
        animation: pulse 0.3s ease-in-out infinite alternate;
    }
    
    @keyframes pulse {
        from { opacity: 0.7; }
        to { opacity: 1; }
    }
    
    .status-text {
        font-size: 1.1em;
        min-height: 30px;
        margin: 15px 0;
        color: #aaa;
    }
    
    .status-text.recording {
        color: #e94560;
        font-weight: 600;
    }
    
    .status-text.success {
        color: #4caf50;
    }
    
    .btn-record {
        padding: 18px 40px;
        font-size: 18px;
        font-weight: 600;
        background: linear-gradient(145deg, #e94560, #c23a51);
        color: white;
        border: none;
        border-radius: 50px;
        cursor: pointer;
        box-shadow: 0 4px 20px rgba(233,69,96,0.4);
        transition: all 0.3s ease;
        margin: 10px;
    }
    
    .btn-record:hover {
        transform: translateY(-2px);
        box-shadow: 0 6px 25px rgba(233,69,96,0.5);
    }
    
    .btn-record.recording {
        background: linear-gradient(145deg, #ff6b6b, #e94560);
        animation: recordPulse 1s ease-in-out infinite;
    }
    
    @keyframes recordPulse {
        0%, 100% { box-shadow: 0 4px 20px rgba(233,69,96,0.4); }
        50% { box-shadow: 0 4px 30px rgba(233,69,96,0.7); }
    }
    
    .btn-stop {
        padding: 18px 40px;
        font-size: 18px;
        font-weight: 600;
        background: linear-gradient(145deg, #4caf50, #388e3c);
        color: white;
        border: none;
        border-radius: 50px;
        cursor: pointer;
        box-shadow: 0 4px 20px rgba(76,175,80,0.4);
        transition: all 0.3s ease;
        margin: 10px;
        display: none;
    }
    
    .btn-stop:hover {
        transform: translateY(-2px);
    }
    
    .hint {
        color: #666;
        font-size: 0.85em;
        margin-top: 20px;
        font-style: italic;
    }
</style>

<div class="clarissa-recorder" id="recorder-ui">
    <h2>🎤 CLARISSA Voice Input</h2>
    <p class="subtitle">Speak to control reservoir simulation</p>
    
    <div class="waveform-container" id="waveform">
        <!-- Waveform bars will be added by JavaScript -->
    </div>
    
    <p class="status-text" id="status">Click to start recording</p>
    
    <button class="btn-record" id="startBtn">🎤 Start Recording</button>
    <button class="btn-stop" id="stopBtn">⏹️ Stop & Process</button>
    
    <p class="hint">💡 Try: "show permeability" or "what is the water cut?"</p>
</div>

<script>
// Initialize waveform bars
(function() {
    const waveform = document.getElementById('waveform');
    for (let i = 0; i < 30; i++) {
        const bar = document.createElement('div');
        bar.className = 'wave-bar';
        bar.id = 'bar-' + i;
        waveform.appendChild(bar);
    }
})();
</script>
'''

VOICE_RECORDER_JS = '''
async function recordWithWaveform() {
    const startBtn = document.getElementById('startBtn');
    const stopBtn = document.getElementById('stopBtn');
    const status = document.getElementById('status');
    const bars = document.querySelectorAll('.wave-bar');
    
    let audioContext, analyser, dataArray, animationId;
    
    // Wait for start button click
    await new Promise(resolve => startBtn.onclick = resolve);
    
    try {
        status.textContent = '🔄 Requesting microphone...';
        status.className = 'status-text';
        
        // Get microphone access
        const stream = await navigator.mediaDevices.getUserMedia({ 
            audio: { channelCount: 1, sampleRate: 16000 } 
        });
        
        // Setup audio analysis for waveform
        audioContext = new AudioContext();
        analyser = audioContext.createAnalyser();
        const source = audioContext.createMediaStreamSource(stream);
        source.connect(analyser);
        analyser.fftSize = 64;
        dataArray = new Uint8Array(analyser.frequencyBinCount);
        
        // Start recording
        const recorder = new MediaRecorder(stream, { mimeType: 'audio/webm' });
        const chunks = [];
        recorder.ondataavailable = e => chunks.push(e.data);
        recorder.start();
        
        // Update UI
        startBtn.style.display = 'none';
        stopBtn.style.display = 'inline-block';
        status.innerHTML = '🔴 <strong>Recording...</strong> Speak now!';
        status.className = 'status-text recording';
        bars.forEach(bar => bar.classList.add('active'));
        
        // Animate waveform
        function animate() {
            analyser.getByteFrequencyData(dataArray);
            bars.forEach((bar, i) => {
                const value = dataArray[i] || 0;
                const height = Math.max(8, value / 3.5);
                bar.style.height = height + 'px';
            });
            animationId = requestAnimationFrame(animate);
        }
        animate();
        
        // Wait for stop button
        await new Promise(resolve => stopBtn.onclick = resolve);
        
        // Stop everything
        cancelAnimationFrame(animationId);
        recorder.stop();
        stream.getTracks().forEach(t => t.stop());
        audioContext.close();
        
        // Reset waveform
        bars.forEach(bar => {
            bar.style.height = '20px';
            bar.classList.remove('active');
        });
        
        status.textContent = '⏳ Processing audio...';
        status.className = 'status-text';
        stopBtn.disabled = true;
        
        // Get audio data
        const blob = await new Promise(resolve => {
            recorder.onstop = () => resolve(new Blob(chunks, { type: 'audio/webm' }));
        });
        
        // Convert to base64
        const reader = new FileReader();
        const base64 = await new Promise(resolve => {
            reader.onloadend = () => resolve(reader.result);
            reader.readAsDataURL(blob);
        });
        
        status.textContent = '✅ Recording complete!';
        status.className = 'status-text success';
        
        return base64;
        
    } catch (err) {
        status.innerHTML = '❌ <strong>Error:</strong> ' + err.message;
        status.className = 'status-text';
        console.error(err);
        return 'ERROR:' + err.message;
    }
}
'''

print("✅ Voice recorder with waveform visualization ready")
print("   Run the next cell to start recording!")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🎙️ RECORD AND PROCESS VOICE COMMAND
# ═══════════════════════════════════════════════════════════════════════════════

def record_voice_command():
    """Record voice with waveform visualization and process with CLARISSA."""
    
    print("═" * 60)
    print("🎤 CLARISSA Voice Input")
    print("═" * 60)
    print()
    
    # Display the recording UI
    display(HTML(VOICE_RECORDER_HTML))
    display(Javascript(VOICE_RECORDER_JS))
    
    # Wait for recording
    print("⏳ Waiting for recording...")
    print("   1. Click 'Start Recording'")
    print("   2. Speak your command")
    print("   3. Click 'Stop & Process'")
    print()
    
    result = eval_js('recordWithWaveform()')
    
    if not result or result.startswith('ERROR:'):
        print(f"❌ Recording failed: {result}")
        return None
    
    # Decode audio
    audio_data = result.split(',')[1]
    audio_bytes = b64decode(audio_data)
    print(f"✅ Recorded {len(audio_bytes):,} bytes of audio")
    
    # Save for playback
    audio_path = '/tmp/clarissa_recording.webm'
    with open(audio_path, 'wb') as f:
        f.write(audio_bytes)
    
    # Playback
    print("\n🔊 Your recording:")
    display(Audio(audio_path))
    
    # Transcribe with Whisper
    print()
    print("─" * 60)
    print("📝 TRANSCRIPTION")
    print("─" * 60)
    
    transcript = None
    if os.getenv('OPENAI_API_KEY'):
        try:
            from openai import OpenAI
            client = OpenAI()
            
            with open(audio_path, 'rb') as f:
                response = client.audio.transcriptions.create(
                    model="whisper-1",
                    file=f,
                    language="en",
                    prompt="Reservoir simulation: permeability, porosity, saturation, pressure, layer, waterflood"
                )
            transcript = response.text
            print(f'🗣️ You said: "{transcript}"')
            
        except Exception as e:
            print(f"⚠️ Whisper error: {e}")
    else:
        print("⚠️ No OPENAI_API_KEY - using demo transcript")
        transcript = "show me the permeability"
        print(f'🗣️ Demo: "{transcript}"')
    
    if not transcript:
        return None
    
    # Parse intent
    print()
    print("─" * 60)
    print("🎯 INTENT PARSING")
    print("─" * 60)
    
    intent = parse_intent(transcript)
    print(f"Intent: {intent.type.value}")
    print(f"Confidence: {intent.confidence:.0%}")
    if intent.slots:
        print(f"Slots: {intent.slots}")
    
    # Execute
    print()
    print("─" * 60)
    print("💬 RESPONSE")
    print("─" * 60)
    
    response = execute_intent(intent)
    print(f"CLARISSA: {response.text}")
    
    if response.visualization:
        print()
        response.visualization.show()
    
    print()
    print("═" * 60)
    
    return response

# Run it!
record_voice_command()

---

## 4️⃣ Text Input Alternative

If microphone doesn't work, use text input:

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 📝 TEXT INPUT DEMO
# ═══════════════════════════════════════════════════════════════════════════════

def process_text_command(text: str):
    """Process a text command through CLARISSA."""
    print(f'📝 Command: "{text}"')
    print()
    
    intent = parse_intent(text)
    print(f"🎯 Intent: {intent.type.value}")
    print(f"📊 Confidence: {intent.confidence:.0%}")
    if intent.slots:
        print(f"📦 Slots: {intent.slots}")
    print()
    
    response = execute_intent(intent)
    print(f"💬 CLARISSA: {response.text}")
    
    if response.visualization:
        response.visualization.show()
    
    return response

# Demo commands
print("═" * 60)
print("📝 TEXT INPUT DEMOS")
print("═" * 60)
print()

In [ ]:
# Demo 1: Show permeability
process_text_command("show me the permeability")

In [ ]:
# Demo 2: Layer cross-section
process_text_command("show layer 3")

In [ ]:
# Demo 3: Query value
process_text_command("what is the water cut?")

In [ ]:
# Demo 4: Water saturation
process_text_command("show water saturation at day 500")

In [ ]:
# Interactive text input
text_input = widgets.Text(
    placeholder='Type a command (e.g., "show porosity")',
    description='Command:',
    layout=widgets.Layout(width='80%')
)

output = widgets.Output()

def on_submit(change):
    with output:
        output.clear_output()
        if text_input.value:
            process_text_command(text_input.value)

text_input.on_submit(lambda x: on_submit(x))
submit_btn = widgets.Button(description="Process", button_style='primary')
submit_btn.on_click(lambda x: on_submit(x))

display(widgets.HBox([text_input, submit_btn]))
display(output)

---

## 5️⃣ SPE Benchmark Tests

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🧪 SPE BENCHMARK TEST SUITE
# ═══════════════════════════════════════════════════════════════════════════════

def run_tests():
    """Run SPE benchmark tests for voice module."""
    
    tests = [
        # General commands
        ("cancel", IntentType.CANCEL, {}),
        ("stop", IntentType.CANCEL, {}),
        ("yes", IntentType.CONFIRM, {}),
        ("help", IntentType.HELP, {}),
        
        # SPE1 - Simple model
        ("show me the pressure distribution", IntentType.VISUALIZE_PROPERTY, {"property": "pressure"}),
        ("display the oil saturation at layer 2", IntentType.VISUALIZE_PROPERTY, {"layer": 2}),
        ("show porosity in 3D", IntentType.VISUALIZE_PROPERTY, {"property": "porosity"}),
        
        # SPE9 - Waterflood
        ("show water saturation at day 500", IntentType.VISUALIZE_PROPERTY, {"property": "water_saturation", "time_days": 500}),
        ("what is the water cut", IntentType.QUERY_VALUE, {"property": "water_cut"}),
        ("display the permeability at layer 8", IntentType.VISUALIZE_PROPERTY, {"property": "permeability", "layer": 8}),
        
        # SPE10 - Large model
        ("show the permeability heterogeneity", IntentType.VISUALIZE_PROPERTY, {"property": "permeability"}),
        ("display porosity at layer 50", IntentType.VISUALIZE_PROPERTY, {"property": "porosity", "layer": 50}),
    ]
    
    print("═" * 65)
    print("🧪 CLARISSA Voice Module - SPE Benchmark Test Suite")
    print("═" * 65)
    print()
    
    passed = 0
    failed = 0
    
    for text, expected_type, expected_slots in tests:
        result = parse_intent(text)
        
        type_ok = result.type == expected_type
        slots_ok = all(result.slots.get(k) == v for k, v in expected_slots.items())
        
        if type_ok and slots_ok:
            print(f"  ✅ '{text[:40]}...'" if len(text) > 40 else f"  ✅ '{text}'")
            passed += 1
        else:
            print(f"  ❌ '{text}'")
            if not type_ok:
                print(f"     └─ Expected {expected_type.value}, got {result.type.value}")
            if not slots_ok:
                print(f"     └─ Expected slots {expected_slots}, got {result.slots}")
            failed += 1
    
    print()
    print("─" * 65)
    if failed == 0:
        print(f"✅ ALL {passed} TESTS PASSED!")
    else:
        print(f"❌ {failed} FAILED, {passed} passed")
    print("═" * 65)
    
    return failed == 0

run_tests()

---

## 📚 Summary

This notebook demonstrated:

1. **🎤 Voice Recording** - Professional waveform visualization with WebAudio API
2. **📝 Speech-to-Text** - Whisper API transcription with domain vocabulary
3. **🎯 Intent Parsing** - Rule-based command understanding
4. **📊 Visualization** - 3D and cross-section reservoir property views
5. **🧪 Testing** - SPE benchmark validation

### Next Steps

- **Full Integration**: Connect to OPM Flow simulations
- **LLM Parsing**: Use Claude/GPT-4 for complex commands
- **Production**: Browser-based standalone demo

---

*CLARISSA - Conversational Language Agent for Reservoir Integrated Simulation System Analysis*